<a href="https://colab.research.google.com/github/stupidboy-dev/IMDB-classification/blob/main/IMDB_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lakshmi25npathi_imdb_dataset_of_50k_movie_reviews_path = kagglehub.dataset_download('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')
tensorflow_bert_tensorflow2_en_uncased_l_12_h_768_a_12_3_path = kagglehub.model_download('tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/3')
tensorflow_bert_tensorflow2_en_uncased_preprocess_3_path = kagglehub.model_download('tensorflow/bert/TensorFlow2/en-uncased-preprocess/3')

print('Data source import complete.')


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
df = pd.read_csv(r"/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv",
                 on_bad_lines="skip",
                engine='python',)

print(df.isna().sum())
print(df.isnull().sum())
df['sentiment'] = df['sentiment'].fillna('unknown')
df.drop(df[df['sentiment'] == 'unknown'].index, inplace=True)
print(df.isna().sum())
print(df.isnull().sum())
df['sentiment'].value_counts()

df['sentiment'] = df['sentiment'].map({"negative":0,"positive":1})

2025-04-20 10:23:13.007730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745144593.211053      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745144593.272160      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


review       0
sentiment    0
dtype: int64
review       0
sentiment    0
dtype: int64
review       0
sentiment    0
dtype: int64
review       0
sentiment    0
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

X = df['review']
y = df.sentiment

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(data=X_train)

data_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
data_train = data_train.batch(64).prefetch(tf.data.AUTOTUNE).cache()

I0000 00:00:1745144611.964093      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
data_train

<CacheDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
import tf_keras as keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Инициализация слоев
preprocessor = hub.KerasLayer(
   "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3",
    name="preprocessing"
)

encoder = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/3",
    trainable=True,
    name="BERT_encoder"
)

# Построение модели
inputs = keras.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(inputs)
outputs = encoder(x)

x = keras.layers.BatchNormalization()(outputs['pooled_output'])
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(32, activation='relu')(x)
predictions = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=[inputs], outputs=[predictions], name='gggg')

model.summary()

Model: "gggg"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                               

In [ ]:
with tf.device('/GPU:0'):
  model.compile(
        optimizer='SGD',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

  model.fit(data_train, epochs=10)

Epoch 1/10


I0000 00:00:1745144698.469937      90 service.cc:148] XLA service 0x7de6ef233fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745144698.470490      90 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745144698.501507      90 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745144698.584859      90 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 648s 980ms/step - loss: 0.3446 - accuracy: 0.8459
Epoch 2/10
625/625 [==============================] - 612s 979ms/step - loss: 0.2751 - accuracy: 0.8834
Epoch 3/10
625/625 [==============================] - 612s 979ms/step - loss: 0.2396 - accuracy: 0.8988
Epoch 4/10
625/625 [==============================] - 612s 979ms/step - loss: 0.2086 - accuracy: 0.9147
Epoch 5/10
625/625 [==============================] - 612s 979ms/step - loss: 0.1801 - accuracy: 0.9262
Epoch 6/10
625/625 [==============================] - 612s 979ms/step - loss: 0.1504 - accuracy: 0.9388
Epoch 7/10
625/625 [==============================] - 612s 979ms/step - loss: 0.1268 - accuracy: 0.9503
Epoch 8/10
625/625 [==============================] - 612s 979ms/step - loss: 0.1067 - accuracy: 0.9582
Epoch 9/10
625/625 [==============================] - 612s 978ms/step - loss: 0.0872 - accuracy: 0.9665
Epoch 10/10
625/625 [==============================] - 611s 978ms/step - lo

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 59s 186ms/step - loss: 0.5084 - accuracy: 0.8789


[0.5083507895469666, 0.8788999915122986]

In [ ]:
text = [
       'Its so goood, the best film with i ever seen']

pred = model.predict(tf.constant(text))


1/1 [==============================] - 0s 24ms/step


In [ ]:
def predict(text:list):
    pred = model.predict(tf.constant(text))
    if round(float(pred)) == 1:
        return 'positive'
    else:
        return 'negative'


ans = predict(['This tea has bright taste'])

1/1 [==============================] - 0s 24ms/step


In [ ]:
ans

'positive'

In [ ]:
sentences = [
    # Позитивные (1)
    "This movie is absolutely fantastic and worth watching!",
    "The service was exceptional, I'm thoroughly impressed.",
    "While the interface feels outdated, the app's reliability and precision make it indispensable for professional use.",
    "I was skeptical due to mixed reviews, but the daring flavor combinations and artistic presentation changed my mind completely.",
    "The novel starts slow but rewards readers with a profound narrative on human resilience—left me intellectually stimulated.",
    "Despite occasional lag during data loads, the app's innovative features save me hours weekly. A game-changer!",
    "The product requires patience to master, but its customizability and 24/7 premium support justify every penny spent.",
    "While not perfect, the developer's transparency in addressing bugs and implementing user feedback is truly commendable.",
    "The workshop blended cutting-edge theory with real-world applications, sparking ideas I've already implemented successfully.",
    "Though pricier than alternatives, the unmatched durability and eco-friendly manufacturing process make this ethical consumerism.",

    # Негативные (0)
    "This is the worst experience I've ever had, terrible service.",
    "The film was boring and poorly directed, a complete waste of time.",
    "The 'smart' features complicate basic tasks—what's advertised as intuitive requires constant tech support to navigate.",
    "A textbook case of style over substance: stunning packaging hides poorly sourced materials and shoddy craftsmanship.",
    "The course content recycled free YouTube tutorials while charging premium prices for 'expert' insights that never materialized.",
    "Five firmware updates later, the device still overheats randomly while failing basic functions it was designed to perform.",
    "Their 'eco-conscious' branding insults intelligence—products arrive wrapped in enough plastic to choke a sea turtle.",
    "The retreat promises serenity but delivers construction noise by day and karaoke parties till midnight. False advertising at its finest.",
    "After 3 warranty 'repairs' and endless support chats, the same critical flaw persists. Now they've stopped responding entirely.",
    "Charges luxury prices for budget motel quality: stained linens, broken AC, and a 'spa' that's literally a converted storage closet."
]

labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pred = []
for text in sentences:
    pred.append(predict([text]))

pred = [1 if t == 'positive' else 0 for t in pred]
print(pred)
print(labels)
pred == labels

1/1 [==============================] - 0s 24ms/step
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


False

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(X_test)

313/313 [==============================] - 59s 185ms/step


TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
from sklearn.metrics import roc_auc_score

predictions = np.round(predictions)
print(classification_report(y_test, predictions))
print(roc_auc_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.82      0.87      4961
           1       0.84      0.94      0.89      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

0.878451905013901


In [ ]:
model.save('/home/archbtw/utils/model.keras')

FileNotFoundError: [Errno 2] No such file or directory: '/home/archbtw/utils/model.keras'